# Summary

`# TODO: summary here`

In [1]:
%load_ext autoreload
%autoreload 3

In [5]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

from htools import *
from incendio.nlp import FillMaskTransform, GenerativeTransform, \
    ParaphraseTransform

/Users/hmamin/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/hmamin/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/hmamin/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/hmamin/anaconda3/lib/python3.7/site-packages/thinc/neural/_custom_kernels.py:36: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/hmamin/anaconda3/lib/python3.7/site-packages/thinc/neural/_custom_kernels.cu' mode='r' encoding='utf8'>
  SRC = (PWD / "_custom_kernels.cu").open("r", encoding="utf8").read()
/Users/hmamin/anaconda3/lib/pyth

In [11]:
cd_root()

Current directory: /Users/hmamin/incendio


In [6]:
TRANSFORMS = {
    'fillmask': FillMaskTransform,
    'paraphrase': ParaphraseTransform,
    'generative': GenerativeTransform
}

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
fm_tfm = TRANSFORMS['fillmask'](n=3)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
@immutify_defaults
def generate(source, dest, transform, n=5, text_col='text', id_cols=(),
             nrows=None, tfm_kwargs={}, call_kwargs={}):
    # Use list args like --id_cols '[col1, col2]'
    # Use dict args like --call_kwargs '{"drop_pct": .2, "min_keep": 4}'
    source, dest = Path(source), Path(dest)

    # For simplicity, we stick to one transform at a time. Slow to load so at
    # least for now, let user pass in the transform itself.
    transform = TRANSFORMS[transform](n=n, **tfm_kwargs) \
        if isinstance(transform, str) else transform
    df = pd.read_csv(source, usecols=[text_col] + list(id_cols), nrows=nrows)
    res = transform(df[text_col].tolist(), **call_kwargs)
    return res, df
    # df_res = transform(df[text_col].tolist())
    # df_res.ends().pprint()
    # df_res.to_csv(dest, index=False)

In [18]:
res, df = generate('/Users/hmamin/msan631/hw3_sentence_pairs/data/train.csv', 
               'data/tmp.csv', fm_tfm, n=3, text_col='question1',
               id_cols=['id', 'qid1'], nrows=5, 
               call_kwargs={'errors': 'warn'})

In [38]:
cols = ['id', 'qid1']
# pd.concat([df[col].repeat(2) for col in ], axis=1)

[[0, 1], [1, 3], [2, 5], [3, 7], [4, 9]]

In [44]:
pd.DataFrame([(sentence, *id_rows) for gen, id_rows 
              in zip(res, df[cols].values.tolist()) for sentence in gen])

,0,1,2
0,What is the step by step guide to invest in sh...,0,1
1,What is the step by step guide investors inves...,0,1
2,What is the step by step guide for invest in s...,0,1
3,What is the story of Kohinoor (Koh-i-Noor) Dia...,1,3
4,Where is the story of Kohinoor (Koh-i-Noor) Di...,1,3
5,What is the story of Kohinoor (Koh-i-Noor) Di...,1,3
6,How can I increase the speed of my internet co...,2,5
7,How can I increase the security of my internet...,2,5
8,How can I increase the reliability of my inter...,2,5
9,Why am I mentally very lonely? How can I solve?,3,7
